# Setup

In [ ]:
%env DEEPFILL=jiahuiyu-deepfill-offline
!cp -r ../input/$DEEPFILL/generative_inpainting_v2 generative_inpainting

In [ ]:
!pip install ../input/$DEEPFILL/*.whl # neuralgym converted with tf_upgrade_v2

In [ ]:
!pip install ../input/tf-slim110/*.whl # used for arg_scope in deepfill lib

In [ ]:
!mkdir checkpoints
!cp -rf /kaggle/input/deepfill-v2-pretrained/places2_256_deepfill_v2 checkpoints

# Imports

In [ ]:
import numpy as np
import cv2
from PIL import Image as pil_image
import tensorflow as tf

# Global

In [ ]:
IMG_SIZE = 512
PROJECT_FOLDER = "../input/hotelid-2022-train-images-512x512/"
TEST_DATA_FOLDER = "../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/test_images/"

## Inpanting

In [ ]:
%cd generative_inpainting
from inpaint_model import InpaintCAModel
import neuralgym as ng

def load_inpaint_model(): # Based on batch_test.py
    FLAGS = ng.Config('inpaint.yml')
    #ng.get_gpus(1)

    sess_config = tf.compat.v1.ConfigProto()
    sess_config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=sess_config)

    model = InpaintCAModel()
    tf.compat.v1.disable_eager_execution()
    input_image_ph = tf.compat.v1.placeholder(
        tf.float32, shape=(1, IMG_SIZE, IMG_SIZE*2, 3))
    output = model.build_server_graph(FLAGS, input_image_ph)
    output = (output + 1.) * 127.5
    output = tf.reverse(output, [-1])
    output = tf.saturate_cast(output, tf.uint8)
    vars_list = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES)
    assign_ops = []
    for var in vars_list:
        vname = var.name
        from_name = vname
        var_value = tf.train.load_variable("../checkpoints/places2_256_deepfill_v2", from_name)
        assign_ops.append(tf.compat.v1.assign(var, var_value))
    sess.run(assign_ops)
    print('Model loaded.')
    return sess, output, input_image_ph

inpaint_model_sess, inpaint_model_output, inpaint_model_ph = load_inpaint_model()
%cd ..

In [ ]:
lower_red = np.array([0,0,200])
upper_red = np.array([55,55,255])
mask_kernel = np.ones((4,4), np.uint8)

def get_mask(img):
    mask = cv2.inRange(img, lower_red, upper_red)
    mask = cv2.dilate(mask, mask_kernel, iterations=1)
    return cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

def inpaint_image(image, mask):
    h, w, _ = image.shape
    grid = 4
    image = image[:h//grid*grid, :w//grid*grid, :]
    mask = mask[:h//grid*grid, :w//grid*grid, :]
    #print('Shape of image: {}'.format(image.shape))

    image = np.expand_dims(image, 0)
    mask = np.expand_dims(mask, 0)
    input_image = np.concatenate([image, mask], axis=2)

    # load pretrained model
    result = inpaint_model_sess.run(inpaint_model_output, feed_dict={inpaint_model_ph: input_image})
    return result[0][:, :, ::-1]

In [ ]:
def pad_image(img):
    w, h, c = np.shape(img)
    if w > h:
        pad = int((w - h) / 2)
        img = cv2.copyMakeBorder(img, 0, 0, pad, pad, cv2.BORDER_CONSTANT, value=0)
    else:
        pad = int((h - w) / 2)
        img = cv2.copyMakeBorder(img, pad, pad, 0, 0, cv2.BORDER_CONSTANT, value=0)
        
    return img

def open_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = pad_image(img)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    mask = get_mask(img)
    img = inpaint_image(img, mask)
    
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
# Test code
from IPython.display import Image
img = open_and_preprocess_image(TEST_DATA_FOLDER+'abc.jpg')
cv2.imwrite("test.png", cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
Image("test.png")

In [ ]:
# More test code
# Add new mask and test again
img = pil_image.open('test.png')
mask = pil_image.open('../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_masks/00095.png')
mask = mask.resize((IMG_SIZE, IMG_SIZE))
img.paste(mask, (0,0), mask)
img.save('test_mask.png')
Image("test_mask.png")

In [ ]:
img = open_and_preprocess_image('test_mask.png')
cv2.imwrite("test2.png", cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
Image("test2.png")